In [ ]:
# type: ignore
import numpy as np

from nanopyx.core.transform.binning import rebin2d
from nanopyx.core.transform.image_magnify import *
from nanopyx.data.examples.download import ExampleDataManager
from nanopyx.core.transform.image_add_random_noise import getRamp, getPerlinNoise

M = 2
downloader = ExampleDataManager()
# z = downloader.get_ZipTiffIterator('LongContinuous_UtrGFP')
z = downloader.get_ZipTiffIterator('SMLMS2013_HDTubulinAlexa647')
#image = z[0]
# image = getRamp(128, 256)
image = getPerlinNoise(64, 128, amp=1000, f=10, octaves=3)

imageDownsampled = rebin2d(image, M, mode="mean")

In [ ]:
%%prun -s cumulative -q -l 100 -T prun_magnify.profile
# Profile the run

imageFFTZoom = fourier_zoom(imageDownsampled.astype(np.float32), M)
imageCRZoom = catmull_rom_zoom(imageDownsampled, M)
imageBCZoom = bicubic_zoom(imageDownsampled, M)
imageLZoom = lanczos_zoom(imageDownsampled, M, taps = 4)
imageScipyZoom = scipy_zoom(imageDownsampled, M)
imageSkiZoom = skimage_zoom(imageDownsampled, M)
imageCVZoom = cv2_zoom(imageDownsampled, M)


interpolated_images = {
    "fft-zoom": imageFFTZoom,
    "catmull-rom-zoom": imageCRZoom,
    "bicubic-zoom": imageBCZoom,
    "lanczos-zoom": imageLZoom,
    "scipy-zoom": imageScipyZoom,
    "skimage-rescale": imageSkiZoom,
    "cv2-resize": imageCVZoom
}

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from nanopyx.core.analysis.pearson_correlation import pearson_correlation
import numpy as np

# Create a figure with 3 subplots arranged in a row
fig, axs = plt.subplots(len(interpolated_images)+1, 2, figsize=(16, 2*16))
sns.heatmap(image, ax=axs[0,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(imageDownsampled, ax=axs[0,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[0,0].set_title("orginal")
axs[0,1].set_title(f"downsampled {M}x")

for i, id in enumerate(interpolated_images):
    interpolated_image = interpolated_images[id]
    error = image-interpolated_image
    rmse = round((error**2).mean()**0.5)
    pc = pearson_correlation(image, interpolated_image)
    sns.heatmap(interpolated_image, ax=axs[i+1,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
    sns.heatmap(error, ax=axs[i+1,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
    axs[i+1,0].set_title(f"{id}")
    axs[i+1,1].set_title(f"{id}-error")
    axs[i+1,1].set_xlabel(f"RMSE={rmse} PC={round(pc, 2)} MaxError={round(np.max(error))}")

# Show the figure
plt.show()
